2. 应用题
   - 使用选择SARSA，期望 SARSA，Q 学习中的一种方法求解小车爬山控制策略 MoutainCar-v0
   - 随机数种子为学号后四位
   - 自行对超参数寻优
   - 训练好后，输出完全利用（epsilon=0）时的回合总奖励（total_reward）（评分比重 60%）
   - 请说明选择该方法的原因（评分比重30%）
   - 将代码和结果整理为ipynb格式（文件名“MTB.ipynb”），需要包含训练过程中每个回合的总奖励变化曲线（评分比重10%）

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import gym

np.random.seed(9999) # 学号后四位

### 环境使用

In [ ]:
env = gym.make('MountainCar-v0')
env.seed(0)
print('观测空间 = {}'.format(env.observation_space))
print('动作空间 = {}'.format(env.action_space))
print('位置范围 = {}'.format((env.unwrapped.min_position, 
        env.unwrapped.max_position)))
print('速度范围 = {}'.format((-env.unwrapped.max_speed,
        env.unwrapped.max_speed)))
print('目标位置 = {}'.format(env.unwrapped.goal_position))

举例控制策略：一直向右

In [ ]:
positions, velocities = [], []
observation = env.reset()
while True:
    positions.append(observation[0])
    velocities.append(observation[1])
    next_observation, reward, done, _ = env.step(2)
    if done:
        break
    observation = next_observation

if next_observation[0] > 0.5:
    print('成功到达')
else:
    print('失败退出')

# 绘制位置和速度图像
fig, ax = plt.subplots()
ax.plot(positions, label='position')
ax.plot(velocities, label='velocity')
ax.legend()